In [3]:
# 递归搜索文件夹下的所有文件，添加到一个列表种
import os

def list_files(res, startpath, startStr, endStr):
    for root, dirs, files in os.walk(startpath):
        for name in files:
            if name.endswith(endStr) and name.startswith(startStr):
                if os.path.join(root, name) not in res:
                    res.append(os.path.join(root, name))
        for dir in dirs:
            list_files(res, os.path.join(root, dir), startStr, endStr)


In [4]:
import win32com.client as win32

# xls 转 xlsx，不会更改任何一点格式


def xls_to_xlsx(xls_file):
  # try:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(xls_file)
    # FileFormat = 51 is for .xlsx extension
    wb.SaveAs(xls_file+"x", FileFormat=51)
    wb.Close(SaveChanges=True)  # FileFormat = 56 is for .xls extension
    excel.Application.Quit()
  # except:
  #   pass


# xls_to_xlsx(r'E:\paper\data\原始数据\中国统计年鉴（1981-2021）\中国统计年鉴2021\zk\html\C23-24.xls')


In [5]:
from openpyxl.styles import PatternFill
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.worksheet.cell_range import CellRange
from openpyxl.cell import Cell
import openpyxl

# 打开 Excel 文件
# wb = openpyxl.load_workbook("C01-02.xlsx")


# 拆分所有的合并单元格，并赋予合并之前的值

def unmerge_and_fill_cells(worksheet: Worksheet) -> None:
    """
    Unmerges all merged cells in the given ``worksheet`` and copies the content
    and styling of the original cell to the newly unmerged cells.

    :param worksheet: The Excel worksheet containing the merged cells.
    """

    # Must convert iterator to list to eagerly evaluate all merged cell ranges
    # before looping over them - this prevents unintended side-effects of
    # certain cell ranges from being skipped since `worksheet.unmerge_cells()`
    # is destructive.
    all_merged_cell_ranges: list[CellRange] = list(
        worksheet.merged_cells.ranges
    )

    for merged_cell_range in all_merged_cell_ranges:
        merged_cell: Cell = merged_cell_range.start_cell
        worksheet.unmerge_cells(range_string=merged_cell_range.coord)

        # Don't need to convert iterator to list here since `merged_cell_range`
        # is cached
        for row_index, col_index in merged_cell_range.cells:
            cell: Cell = worksheet.cell(row=row_index, column=col_index)
            cell.value = ''.join(str(merged_cell.value).split())

            # (Optional) If you want to also copy the original cell styling to
            # the newly unmerged cells, you must use shallow `copy()` since
            # cell style properties are proxy objects which are not hashable.
            #
            # See <https://openpyxl.rtfd.io/en/stable/styles.html#copying-styles>
            # from copy import copy
            # cell.alignment = copy(merged_cell.alignment)
            # cell.border = copy(merged_cell.border)
            # cell.font = copy(merged_cell.font)
            cell.fill = PatternFill(fgColor='ff6600',fill_type='solid') # 定义合并拆分单元格背景色

# for sheet in wb.worksheets:
#     unmerge_and_fill_cells(sheet)
    
# wb.save("C01-02-split.xlsx")


In [6]:
import json




In [54]:
from openpyxl import load_workbook
import os
import ntpath
import json

import re


def find_year(text):
    pattern = r'(1[89]\d{2}|20[0-9][0-9])'
    year = re.findall(pattern, text)
    if year:
      return year[-1]
    else:
      return None

# provinces = ['中国', '北京', '天津', '河北', '山西', '内蒙古', '辽宁', '吉林', '黑龙江',
#              '上海', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南', '湖北',
#              '湖南', '广东', '广西', '海南', '重庆', '四川', '贵州', '云南',
#              '西藏', '陕西', '甘肃', '青海', '宁夏', '新疆', '台湾', '香港', '澳门']


cities = []
with open('省市县地名.json', 'r', encoding='utf-8') as file:
    cities = json.load(file)
cities.append('中国')
cities.append('全国')
def contains_city(string):
    for city in cities:
        if re.search(city, string):
            return city
    return None


def find_unit(text):
    # pattern = r'\b\d+(?:[\.\,]\d+)?\s*(?:亿元|万元|元|万人|人|%|亿吨|万吨|万台|万辆|万千瓦|亿千瓦小时|亿立方米|万平方米|百万人次|个|千克|千瓦小时)\b'
    pass_list = ['年末']
    matches_chinese = re.findall(r'\（([\u4e00-\u9fff]+)\）', text)
    matches_unit = re.findall(r'单位：(\w+)', text)
    matches_percent = re.findall(r'\((%%)\)', text)
    if len(matches_chinese) or len(matches_percent)  or len(matches_unit):
      temp = matches_chinese + matches_unit + matches_percent
      if temp[-1] in pass_list:
        return None
      return temp[-1]
    else:
      return None

def handle_file(file, output_tree_dir, output_value_dir, result_value):
  file_year = find_year(file)
  file_space = contains_city(file)
  yearbook = re.search(r'中国统计年鉴\d{4}', file)[0]
  wb = load_workbook(file)
  file_name = ntpath.basename(file).split('.')[0]

  for sheet in wb.worksheets:
    # 单个表的转变数据
    merged_rows = set()
    for i in sheet.merged_cell_ranges:
      for j in range(i.min_row, i.max_row + 1):
        merged_rows.add(j)

    rows = sheet.max_row
    cols = sheet.max_column

    # 划分区域
    title_index = []
    title_flag = True

    main_index = []
    note_index = []

    for index in range(1, rows):
      non_empty_cell = set()
      for cell in sheet[index]:
        if cell.value:
          non_empty_cell.add(cell.value)
      if title_flag and len(non_empty_cell) == 1:
        title_index.append(index)
        title_flag = False
      if title_flag and len(non_empty_cell) == 1:
        note_index.append(index)

      if len(non_empty_cell) > 1:
        main_index.append(index)
        break

    for index in reversed(range(1, rows + 1)):
      non_empty_cell = set()
      for cell in sheet[index]:
        if cell.value:
          non_empty_cell.add(cell.value)
      if len(non_empty_cell) == 1:
        note_index.append(index)
      if len(non_empty_cell) > 1:
        main_index.append(index)
        break
        
    # 从 main_index 中得到行列表 row_index, 列列表 col_index, 数值盒状区
    row_index = []
    col_index = []
    value_index = {}

    if len(main_index) == 0 :
      return
    for index in range(main_index[0], main_index[1] + 1):
      not_number_value_count = 0
      all_empty = all(cell.value is None or cell.value ==
                      '' or isinstance(cell.value, str) and cell.value.isspace() for cell in sheet[index])
      year_numbers = 0
      value_numbers = 0
      float_numbers = 0
      if index in merged_rows:
        continue
      for cell in sheet[index]:
        if not isinstance(cell.value, (int, float)):
          not_number_value_count += 1
        if isinstance(cell.value, int) and cell.value > 1900 and cell.value < 2300:
          year_numbers += 1
        if isinstance(cell.value, int) or isinstance(cell.value, float):
          value_numbers += 1
        if isinstance(cell.value, float):
          float_numbers += 1

      if all_empty or not_number_value_count == 1 and year_numbers != cols-1 or value_numbers >= cols-1 or float_numbers >= 1:
        row_index.append(main_index[0])
        row_index.append(index - 1)
        col_index.append(1)
        value_index[0] = index
        value_index[2] = main_index[1]
        value_index[3] = cols
        break

    for index in range(1, cols):
      no_str_flag = all(not isinstance(sheet.cell(row=row_index, column=index), str)
                        for row_index in range(value_index.get(0), value_index.get(2)))
      if(no_str_flag):
        value_index[1] = index + 1
        col_index.append(index)
        break

    # print('row_index', row_index)
    # print('col_index', col_index)
    # print('value_index', value_index)

    # 语义树，根据，单元格合并，单元格加粗，等方式来识别语义树，子树用 children

    # 列表头处理, 目前就处理加粗表格这一种
    font_cells = []
    col_tree = []
    for col in range(col_index[0], col_index[1] + 1):
      for row in range(value_index[0], value_index[2]):
        if sheet.cell(row, col).font.bold:
          font_cells.append([row, col])
    # font_cells = [sheet.cell(row, col).font.bold for col in range(col_index[0], col_index[1]) for row in range(value_index[0], value_index[2])]
    i = value_index[2] + 1
    for index in range(len(font_cells)):
      if not isinstance(sheet.cell(font_cells[index][0], font_cells[index][1]).value, str):
        continue
      key = ''.join(sheet.cell(
          font_cells[index][0], font_cells[index][1]).value.split())
      temp = {}
      temp['value'] = key
      i = font_cells[index][0] + 1
      while index + 1 < len(font_cells) and i < font_cells[index + 1][0] and i < value_index[2]:
        if 'children' not in temp:
          temp['children'] = []
        if isinstance(sheet.cell(i, font_cells[index][1]).value, str):
          temp['children'].append(
            ''.join(sheet.cell(i, font_cells[index][1]).value.split()))
        i += 1
      col_tree.append(temp)

    while i <= value_index[2]:
      if 'children' not in temp:
        temp['children'] = []
      if isinstance(sheet.cell(i, font_cells[index][1]).value, str):
          temp['children'].append(
              ''.join(sheet.cell(i, font_cells[index][1]).value.split()))
      i += 1

    # print('col_tree', col_tree)

    # 接下来处理行表头，目前就合并单元格这一种,已经将合并单元格分开，就用set来处理合并单元格相同的值

    row_tree = []
    for row in reversed(range(row_index[0], row_index[1] + 1)):
      for col in range(1, cols + 1):
        i = row
        stack = []
        while i >= row_index[0]:
          while i > row_index[0] and sheet.cell(i, col).value == sheet.cell(i-1, col).value:
            i -= 1
          stack.append(sheet.cell(i, col).value)
          i -= 1

        preNode = row_tree
        if len(stack) > 0:
          while len(stack) > 0:
            isMeet = False
            for i in preNode:
              if len(stack) > 0 and stack[len(stack) - 1] == i['value']:
                isMeet = True
                stack.pop()
                if 'children' in i:
                  preNode = i['children']
                break
            if not isMeet:
              break

        while len(stack) > 0:
          temp = {}
          temp['value'] = stack[len(stack) - 1]
          if len(stack) > 1:
            temp['children'] = []
          preNode.append(temp)
          if 'children' in temp:
            preNode = temp['children']
          stack.pop()


    # 表头
    title_data = sheet.cell(title_index[0], 1).value

    # 注释值
    note_data = []
    for item in note_index:
      if isinstance(sheet.cell(item, 1).value, str) or isinstance(sheet.cell(item, 1).value, int):
        note_data.append(''.join(str(sheet.cell(item, 1).value).split()))

    # 现在将数值盒装区的存在一条一条的记录
    value_data = []
    for row in range(value_index[0], value_index[2] + 1):
      for col in range(value_index[1], value_index[3] + 1):
        if isinstance(sheet.cell(row, col).value, int) or isinstance(sheet.cell(row, col).value, float):
          temp = {}
          temp['value'] = sheet.cell(row, col).value
          # 现在应该从中挖掘时间、空间、量纲
          time = []
          time.append(file_year)
          space = []
          space.append(file_space)
          index_value = []
          unit = []

          for i in range(row_index[0], row_index[1] + 1):
            temp_value = str(sheet.cell(i, col).value)
            temp_value = str.strip(temp_value)
            # if isinstance(temp_value, str) or isinstance(temp_value, int):
            if temp_value not in index_value:
              index_value.append(temp_value)
            

          for j in range(col_index[0], col_index[1] + 1):
            if isinstance(sheet.cell(row, j).value, str) or isinstance(sheet.cell(row, j).value, int):
              temp_value = ''.join(str(sheet.cell(row, j).value).split())
            temp_value = str.strip(temp_value)
            # if isinstance(temp_value, str) or isinstance(temp_value, int):
            if temp_value not in index_value:
              index_value.append(temp_value)
            temp_unit = find_unit(temp_value)
            if temp_unit:
              unit.append(temp_unit)
            else:
              temp_row = row - 1
              while temp_row > value_index[0]:
                temp_unit1 = find_unit(str(sheet.cell(temp_row, j).value))
                if temp_unit1:
                  unit.append(temp_unit1)
                  break
                temp_row -= 1

          for i in index_value:
            temp_time = find_year(i)
            if temp_time and temp_time not in time:
              time.append(temp_time)
            temp_space = contains_city(i)
            if temp_space and temp_space not in space:
              space.append(temp_space)
            temp_unit = find_unit(i)
            if temp_unit:
              unit.append(temp_unit)

          temp['time'] = time[-1]
          temp['space'] = space[-1]
          temp['index'] = index_value
          temp['unit'] = unit
          temp['source'] = yearbook + ' ' + title_data
          temp['note'] = note_data
          value_data.append(temp)
  
          
    treeJson = output_tree_dir + '/' + file_name + '_' + sheet.title + '_tree.json'
    temp = {}
    temp['row_tree'] = row_tree
    temp['col_tree'] = col_tree
    with open(treeJson, 'w', encoding='utf-8') as f:
      json.dump(temp, f, ensure_ascii=False, indent=4)
    valueJson = output_value_dir + '/' + file_name + '_' + sheet.title + '_value.json'
    result_value += value_data
    with open(valueJson, 'w', encoding='utf-8') as f:
      json.dump(value_data, f, ensure_ascii=False, indent=4)



# handle_file(
#     r"F:\Download\第二期 34省统计年鉴\2001-2021各省\2021省统计年鉴\江苏统计年鉴2021\nj21\nj21split.xlsx", os.getcwd(), os.getcwd() + '/temp', [])


In [51]:
origin_file = []

list_files(origin_file,
           r'E:\paper\data\原始数据\中国统计年鉴output\中国统计年鉴2021\splitXlsx', '', 'split.xlsx')


In [55]:

fail_json = []
result = []
output_dir = r'E:\paper\data\原始数据\中国统计年鉴output\中国统计年鉴2021'
for item in origin_file:
  try:
  #   # yearbook = re.search(r'中国统计年鉴\d{4}', item)[0]
  #   # yearbook_dir = output_dir + '/' + yearbook
  #   file_name = file_name = ntpath.basename(item).split('.')[0]
  #   valueJson = output_dir+'/valueJson' + '/' + file_name + '_value.json'
  #   if not os.path.exists(valueJson) or os.path.getsize(valueJson) == 0:
    handle_file(item, output_tree_dir=output_dir + '/treeJson',
                output_value_dir=output_dir+'/valueJson', result_value=result)
  except:
    fail_json.append(item)
    print(item)


C:\Users\wutian\AppData\Local\Temp\ipykernel_34196\2411765132.py:59: DeprecationWarning: Call to deprecated function merged_cell_ranges (Use ws.merged_cells.ranges).
  for i in sheet.merged_cell_ranges:


E:\paper\data\原始数据\中国统计年鉴output\中国统计年鉴2021\splitXlsx\C16-20split.xlsx


In [47]:
with open('江苏省fail.json', 'w', encoding='utf-8') as f:
    json.dump(fail_json, f, ensure_ascii=False, indent=4)
with open('江苏省result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)


In [48]:
len(result)

74069